<a href="https://colab.research.google.com/github/sosier/Learning/blob/main/Pytorch_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Pytorch Linear Regression**

In [1]:
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {DEVICE}")

Using cuda


In [2]:
train_data_path = "sample_data/california_housing_train.csv"
test_data_path = "sample_data/california_housing_test.csv"
y_column = "median_house_value"

## **Load Data**

In [3]:
train = pd.read_csv(train_data_path)
train

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0
...,...,...,...,...,...,...,...,...,...
16995,-124.26,40.58,52.0,2217.0,394.0,907.0,369.0,2.3571,111400.0
16996,-124.27,40.69,36.0,2349.0,528.0,1194.0,465.0,2.5179,79000.0
16997,-124.30,41.84,17.0,2677.0,531.0,1244.0,456.0,3.0313,103600.0
16998,-124.30,41.80,19.0,2672.0,552.0,1298.0,478.0,1.9797,85800.0


In [4]:
test = pd.read_csv(test_data_path)
test

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.05,37.37,27.0,3885.0,661.0,1537.0,606.0,6.6085,344700.0
1,-118.30,34.26,43.0,1510.0,310.0,809.0,277.0,3.5990,176500.0
2,-117.81,33.78,27.0,3589.0,507.0,1484.0,495.0,5.7934,270500.0
3,-118.36,33.82,28.0,67.0,15.0,49.0,11.0,6.1359,330000.0
4,-119.67,36.33,19.0,1241.0,244.0,850.0,237.0,2.9375,81700.0
...,...,...,...,...,...,...,...,...,...
2995,-119.86,34.42,23.0,1450.0,642.0,1258.0,607.0,1.1790,225000.0
2996,-118.14,34.06,27.0,5257.0,1082.0,3496.0,1036.0,3.3906,237200.0
2997,-119.70,36.30,10.0,956.0,201.0,693.0,220.0,2.2895,62000.0
2998,-117.12,34.10,40.0,96.0,14.0,46.0,14.0,3.2708,162500.0


In [5]:
class Normalizer():
  def __init__(self, train_df):
    X = train_df.drop(y_column, axis=1)
    y = train_df[y_column]

    self.X_mean = X.mean()
    self.X_std = X.std()

    self.y_mean = y.mean()
    self.y_std = y.std()
  
  def X(self, X_df):
    return (X_df - self.X_mean)/self.X_std
  
  def y(self, y_df):
    return (y_df - self.y_mean)/self.y_std

def get_X(df, add_intercept=True, normalizer=None, **kwargs):
  X = df.drop(y_column, axis=1)
  
  if normalizer:
    X = normalizer.X(X)

  X = torch.tensor(X.values)

  if add_intercept:
    X = F.pad(X, (1, 0), value=1)  # Add column of ones on left for y-intercept (beta_0)
  
  return X.to(DEVICE)

def get_y(df, normalizer=None, **kwargs):
  y = df[y_column]
  if normalizer:
    y = normalizer.y(y)

  return torch.tensor(y).to(DEVICE)

def get_X_y(df, **kwargs):
  return get_X(df, **kwargs), get_y(df, **kwargs)

In [6]:
X, y = get_X_y(train)
X_test, y_test = get_X_y(test)

normalizer = Normalizer(train)
X_norm, y_norm = get_X_y(train, normalizer=normalizer)
X_norm_test, y_norm_test = get_X_y(test, normalizer=normalizer)

print("Unnormalized:")
print(X.shape, y.shape, X_test.shape, y_test.shape)

print("\nNormalized:")
print(X_norm.shape, y_norm.shape, X_norm_test.shape, y_norm_test.shape)

Unnormalized:
torch.Size([17000, 9]) torch.Size([17000]) torch.Size([3000, 9]) torch.Size([3000])

Normalized:
torch.Size([17000, 9]) torch.Size([17000]) torch.Size([3000, 9]) torch.Size([3000])


## **Perform Regression**

### **Approach 1: Closed-Form Linear Algebra Solution**

**Linear Regression Formula Derivation:**
$$ \vec{y} = X\vec{\beta} + ϵ $$
$$ X^{T}\vec{y} = X^{T}X \vec{\beta} $$
$$ (X^{T}X)^{-1} X^{T}\vec{y} = (X^{T}X)^{-1} X^{T}X \vec{\beta} $$
$$ \vec{\beta} = (X^{T}X)^{-1} X^{T}\vec{y} $$

In [9]:
model_1 = torch.inverse(X.T @ X) @ X.T @ y
print(model_1.device)
print(model_1.shape)
print(model_1)

cuda:0
torch.Size([9])
tensor([-3.6206e+06, -4.3140e+04, -4.2926e+04,  1.1507e+03, -8.3783e+00,
         1.1765e+02, -3.8489e+01,  4.5436e+01,  4.0507e+04], device='cuda:0',
       dtype=torch.float64)


### **Approach 2: Fit Regression Using Stochastic Gradient Descent (SGD)**

In [10]:
class LinearRegression(nn.Module):
  def __init__(self):
      super(LinearRegression, self).__init__()
      self.regression = nn.Sequential(
        nn.Linear(X.shape[1], 1, bias=False, dtype=torch.float64)
      )

  def forward(self, X):
      return self.regression(X)

def train(
    model,
    X, y,
    X_test, y_test,
    batch_size=512,
    batches_to_run=500,
    learning_rate=1e-8,
    print_every=100
):
  loss_fn = nn.MSELoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

  for i in range(batches_to_run):
    # Put model into training mode:
    model.train()

    # Get current batch data
    samples_ids = torch.randint(len(X), size=(batch_size, ))
    train_X = X[samples_ids]
    train_y = y[samples_ids]

    # Do forward pass and evaluate loss
    predicted = model(train_X).view(train_y.shape)
    loss = loss_fn(predicted, train_y)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Reporting
    if i % print_every == 0:
        model.eval()
        predicted = model(X_test).view(y_test.shape)
        loss = loss_fn(predicted, y_test)
        print("loss =", loss.item())
        print("")


#### **2a: Unnormalized Data**

In [11]:
model_2a = LinearRegression().to(DEVICE)
print(model_2a)
print(list(model_2a.parameters()))

LinearRegression(
  (regression): Sequential(
    (0): Linear(in_features=9, out_features=1, bias=False)
  )
)
[Parameter containing:
tensor([[ 0.1618, -0.1895,  0.2011,  0.1831, -0.2902,  0.3228, -0.0247,  0.0460,
          0.3112]], device='cuda:0', dtype=torch.float64, requires_grad=True)]


In [12]:
train(
    model_2a,
    X, y,
    X_test, y_test,
    batches_to_run=int(1e6),
    batch_size=1028,
    learning_rate=1e-8,
    print_every=int(1e5)
  )
print(list(model_2a.parameters()))

loss = 40193255770.050446

loss = 10747022750.080841

loss = 10649356211.997507

loss = 10565572319.485956

loss = 10499867650.904354

loss = 10431759703.426239

loss = 10371799309.485222

loss = 10324760191.661005

loss = 10258433303.174356

loss = 10205132744.170042

[Parameter containing:
tensor([[   12.7302, -1342.2616,  -563.9280,  1532.7026,    42.5374,  -270.3183,
           -64.0836,   276.8620,  2300.8707]], device='cuda:0',
       dtype=torch.float64, requires_grad=True)]


#### **2b: Normalized Data**

In [13]:
model_2b = LinearRegression().to(DEVICE)
print(model_2b)
print(list(model_2b.parameters()))

LinearRegression(
  (regression): Sequential(
    (0): Linear(in_features=9, out_features=1, bias=False)
  )
)
[Parameter containing:
tensor([[ 0.1159,  0.0454, -0.0047, -0.1596,  0.3159, -0.2514, -0.1080,  0.0377,
          0.0828]], device='cuda:0', dtype=torch.float64, requires_grad=True)]


In [14]:
train(
    model_2b,
    X_norm, y_norm,
    X_norm_test, y_norm_test,
    batches_to_run=50000,
    batch_size=1028,
    learning_rate=1e-3,
    print_every=5000
  )
print(list(model_2b.parameters()))

loss = 0.8897616168117417

loss = 0.388080712823599

loss = 0.3680945476231335

loss = 0.3637589560521053

loss = 0.3626976776065821

loss = 0.36225325603410163

loss = 0.3620901580641107

loss = 0.3619936334702206

loss = 0.36190425559789885

loss = 0.36174870602895765

[Parameter containing:
tensor([[ 2.2124e-04, -7.3706e-01, -7.8474e-01,  1.2455e-01, -1.4074e-01,
          3.4071e-01, -3.9707e-01,  2.3927e-01,  6.6125e-01]], device='cuda:0',
       dtype=torch.float64, requires_grad=True)]


## **Evaluate Results**

In [15]:
baseline_prediction = y.mean()
print("Baseline RMSE:", torch.sqrt(torch.mean((baseline_prediction - y_test)**2)).item())
print("Baseline MAE:", torch.mean(torch.abs(baseline_prediction - y_test)).item())
print("")

y_hat = X_test @ model_1
print("Model 1 RMSE:", torch.sqrt(torch.mean((y_hat - y_test)**2)).item())
print("Model 1 MAE:", torch.mean(torch.abs(y_hat - y_test)).item())
print("")

model_2a.eval()
y_hat = model_2a(X_test).squeeze()  # n x 1 to n
print("Model 2a RMSE:", torch.sqrt(torch.mean((y_hat - y_test)**2)).item())
print("Model 2a MAE:", torch.mean(torch.abs(y_hat - y_test)).item())
print("")

model_2b.eval()
y_hat = model_2b(X_norm_test).squeeze()  # n x 1 to n
y_hat = y_hat * normalizer.y_std + normalizer.y_mean  # De-normalize y
print("Model 2b RMSE:", torch.sqrt(torch.mean((y_hat - y_test)**2)).item())
print("Model 2b MAE:", torch.mean(torch.abs(y_hat - y_test)).item())

Baseline RMSE: 113110.18658146849
Baseline MAE: 89731.04773882353

Model 1 RMSE: 69765.36022218474
Model 1 MAE: 50352.22825805612

Model 2a RMSE: 100754.68240539066
Model 2a MAE: 78390.4515916131

Model 2b RMSE: 69766.8635255155
Model 2b MAE: 50338.97467580825
